<a href="https://colab.research.google.com/github/kararmirza19/kararmirza19/blob/main/Movies_Recommender__Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

data = pd.read_csv("/content/filmtv_movies.csv", encoding="utf-8", error_bad_lines=False)
data



<ipython-input-46-0af1bbddf2a5>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("/content/filmtv_movies.csv", encoding="utf-8", error_bad_lines=False)
Skipping line 1338: expected 19 fields, saw 20



,filmtv_id,title,year,genre,duration,country,directors,actors,avg_vote,critics_vote,public_vote,total_votes,description,notes,humor,rhythm,effort,tension,erotism
0,2,Bugs Bunny's Third Movie: 1001 Rabbit Tales,1982,Animation,76,United States,"David Detiege, Art Davis, Bill Perez",NaN,7.7,8.00,7.0,22,"With two protruding front teeth, a slightly sl...","These are many small independent stories, whic...",3,3,0,0,0.0
1,3,18 anni tra una settimana,1991,Drama,98,Italy,Luigi Perelli,"Kim Rossi Stuart, Simona Cavallari, Ennio Fant...",6.5,6.00,7.0,4,"Samantha, not yet eighteen, leaves the comfort...","Luigi Perelli, the director of the ""Piovra"", o...",0,2,0,2,0.0
2,17,Ride a Wild Pony,1976,Romantic,91,United States,Don Chaffey,"Michael Craig, John Meillon, Eva Griffith, Gra...",5.7,6.00,5.0,10,"In the Australia of the pioneers, a boy and a ...","""Ecological"" story with a happy ending, not wi...",1,2,1,0,0.0
3,18,Diner,1982,Comedy,95,United States,Barry Levinson,"Mickey Rourke, Steve Guttenberg, Ellen Barkin,...",7.0,8.00,6.0,18,Five boys from Baltimore have a habit of meeti...,A cast of will be famous for Levinson's direct...,2,2,0,1,2.0
4,20,A che servono questi quattrini?,1942,Comedy,85,Italy,Esodo Pratelli,"Eduardo De Filippo, Peppino De Filippo, Clelia...",5.9,5.33,7.0,15,"With a stratagem, the penniless and somewhat p...",Taken from the play by Armando Curcio that the...,3,1,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36872,232817,Gold Digger Killer,2021,Thriller,87,"Canada, United States",Robin Hays,"Julie Benz, Roan Curtis, Georgia Bradner, Eli ...",4.0,NaN,4.0,3,Celeste is an attractive waitress in the forti...,Freely taken from a true story.,0,0,0,0,0.0
36873,232893,Addio al nubilato 2,2023,Comedy,90,Italy,Francesco Apolloni,"Laura Chiatti, Chiara Francini, Antonia Liskov...",2.7,NaN,3.0,3,"When Eleonora is downloaded to the altar, her ...",Bachelorette fare sequel (2021).,0,0,0,0,0.0
36874,232915,Konferensen,2023,Horror,100,Sweden,Patrik Eklund,"Katia Winter, Eva Melander, Lola Zackow, Adam ...",6.0,NaN,6.0,6,A team building conference organized for a gro...,NaN,0,0,0,0,0.0
36875,232919,Ballelina,2023,Thriller,92,South Korea,Chung-Hyun Lee,"Jeon Jong-seo, Park Yu-rim, Ji-hun Kim",5.8,NaN,6.0,5,Once Ok-Ju worked as a bodyguard and was one o...,NaN,0,0,0,0,0.0


In [ ]:
# Checking and handling missing values

data.isnull().sum()
data.dropna(inplace=True)

data

,filmtv_id,title,year,genre,duration,country,directors,actors,avg_vote,critics_vote,public_vote,total_votes,description,notes,humor,rhythm,effort,tension,erotism
1,3,18 anni tra una settimana,1991,Drama,98,Italy,Luigi Perelli,"Kim Rossi Stuart, Simona Cavallari, Ennio Fant...",6.5,6.00,7.0,4,"Samantha, not yet eighteen, leaves the comfort...","Luigi Perelli, the director of the ""Piovra"", o...",0,2,0,2,0.0
2,17,Ride a Wild Pony,1976,Romantic,91,United States,Don Chaffey,"Michael Craig, John Meillon, Eva Griffith, Gra...",5.7,6.00,5.0,10,"In the Australia of the pioneers, a boy and a ...","""Ecological"" story with a happy ending, not wi...",1,2,1,0,0.0
3,18,Diner,1982,Comedy,95,United States,Barry Levinson,"Mickey Rourke, Steve Guttenberg, Ellen Barkin,...",7.0,8.00,6.0,18,Five boys from Baltimore have a habit of meeti...,A cast of will be famous for Levinson's direct...,2,2,0,1,2.0
4,20,A che servono questi quattrini?,1942,Comedy,85,Italy,Esodo Pratelli,"Eduardo De Filippo, Peppino De Filippo, Clelia...",5.9,5.33,7.0,15,"With a stratagem, the penniless and somewhat p...",Taken from the play by Armando Curcio that the...,3,1,1,0,0.0
5,21,The Uranian Conspiracy,1978,Spy,117,"Italy, Germany, Israel","Gianfranco Baldanello, Menahem Golan","Fabio Testi, Janet Agren, Assaf Dayan, Siegfri...",4.8,3.50,6.0,5,Two Israeli secret agents discover that traffi...,"Action and chases for half of Europe, espionag...",1,2,0,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36760,224974,Il fascino dell'insolito: Impostore,1981,Sci-Fi,52,Italy,"Andrea Frazzi, Antonio Frazzi","Daniele Dublino, Adalberto Maria Merli, Antoni...",4.8,4.50,5.0,3,An alien spatial ship arrives on Earth and a c...,From a story by Philip K. Dick.,0,0,0,0,0.0
36769,225477,"She Is Me, I Am Her",2022,Drama,70,Japan,Mayu Nakamura,"Nahana, Fusako Urabe, Kôta Kusano, Masao Yoshi...",6.5,7.00,6.0,3,A phone call between three former friends of t...,Film that tells human relationships during the...,0,0,0,0,0.0
36795,226388,The Exorcist: Believer,2023,Horror,121,United States,David Gordon Green,"Ellen Burstyn, Leslie Odom Jr., Ann Dowd, Raph...",4.1,4.00,4.0,17,A 12 -year -old girl is owned by a mysterious ...,Sequel to the exorcist.,0,0,0,0,0.0
36798,226639,Sambizanga,1972,Drama,97,"Angola, France",Sarah Maldoror,"Elisa Andrade, Jean M'Vondo, Dino Abelino, Ben...",7.5,7.50,8.0,4,"Angola, 1961. Domingos Xavier, an anti -coloni...",Taken from a true story told in the book in Vi...,0,0,0,0,0.0


In [ ]:
#  Building the Recommendation System
# Create a feature matrix using CountVectorizer:
# Calculation of cosine similarity between movies:

vector = CountVectorizer()
features = vector.fit_transform(data["description"])
similarity_matrix = cosine_similarity(features)

print(similarity_matrix)

[[1.         0.58669926 0.36792475 ... 0.38323816 0.50789938 0.40470392]
 [0.58669926 1.         0.31525772 ... 0.47775452 0.56280925 0.4578    ]
 [0.36792475 0.31525772 1.         ... 0.2574121  0.29240925 0.21746411]
 ...
 [0.38323816 0.47775452 0.2574121  ... 1.         0.38112741 0.31887407]
 [0.50789938 0.56280925 0.29240925 ... 0.38112741 1.         0.46101675]
 [0.40470392 0.4578     0.21746411 ... 0.31887407 0.46101675 1.        ]]


In [ ]:

# Define a function to recommend movies based on a target movie:

def recommend_movies(movie_id, top_n=10):
    similar_movies = np.argsort(similarity_matrix[movie_id])[1:top_n+1]
    recommended_movies = data.iloc[similar_movies][["title", "year", "genre"]].values.tolist()
    return recommended_movies



In [ ]:
target_movie = data[data["title"] == "The Matrix"].index[0]

In [ ]:
#Displaying Top 10 Recommendations from the data set

recommendations = recommend_movies(target_movie)

print(f"Top 10 recommended movies for The Matrix:\n")
for i, movie in enumerate(recommendations, start=1):
    title, year, genre = movie
    print(f"{i}. {title} ({year})  Genre {genre}")


Top 10 recommended movies for The Matrix:

1. Der Dirnenmörder von London (1976)  Genre Drama
2. Taare Zameen Par  (2007)  Genre Drama
3. Ringo e Gringo contro tutti (1966)  Genre Comedy
4. Fair Play (2006)  Genre Thriller
5. Quattro bravi ragazzi (1993)  Genre Drama
6. Radio (2003)  Genre Drama
7. A ridosso dei ruderi, i Trionfi (1997)  Genre Sperimental
8. Papa Luciani. Il sorriso di Dio (2006)  Genre Drama
9. Hwajang (2014)  Genre Drama
10. Hanjo mugen jigoku (1970)  Genre Drama
